# Aim

Analyzing Google Play and App Store data to help developers understand which Apps are most likely to attract more users. The developers in this scenario work on a non-paid mobile Application that monetizes via in-app ads and targets Enlish-speaking users. 

The two datasets that are being analysed can be directly downloaded [here](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv) (Google Play store data) and [here](https://dq-content.s3.amazonaws.com/350/AppleStore.csv) (App Store data).

# Opening and exploring the data

In [1]:
from csv import reader

def csv2list(csv):
    file = open(csv)
    read = reader(file)
    data = list(read)
    return data

google = csv2list("googleplaystore.csv")
apple = csv2list("AppleStore.csv")

In [2]:
def explore(csv, start, end):
    sliced = csv[start:end]

    ll = len(csv)
    
    print(f'The data has {ll} rows')
    print('\n')
    
    for i in sliced:
        print(i)
        print('\n')

In [3]:
print(">>> GOOGLE DATA")
explore(google,0,3)

>>> GOOGLE DATA
The data has 10842 rows


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']




In [4]:
print(">>> APPLE DATA")
explore(apple,0,3)

>>> APPLE DATA
The data has 7198 rows


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']




# Data cleaning 1 - Checking for inacurate data

In [5]:
def row_check(data):
    header = len(data[0])
    
    for i in data:        
        if len(i) != header:
            print(i)
            print(data.index(i))
                  
print(">>> Google check results ")
row_check(google)

print(">>> Apple check results")
row_check(apple)

>>> Google check results 
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10473
>>> Apple check results


In [6]:
del google[10473]

# Data cleaning 2 - Checking for duplicates

Checking for duplicate entries in both datasets via a custom-function, thats flexible enough to be applied on both datasets, who show a different formatting.

In [7]:
def duplicate_check(data, column):
    
    duplicates = []
    unique = []
    
    for i in data:
        name = i[column]
        
        if name in unique:
            duplicates.append(name)
        else:
            unique.append(name)
    
    return duplicates

In [8]:
google_duplicates = duplicate_check(google, 0)
apple_duplicates = duplicate_check(apple, 1)

print(">>> Examples of duplicates within Google dataset:")
print(google_duplicates[:10])
print(">>> Count of duplicates: " + str(len(google_duplicates)))
print("\n")
print(">>> Examples of duplicates within Apple dataset:")
print(apple_duplicates[:10])
print(">>> Count of duplicates: " + str(len(apple_duplicates)))

>>> Examples of duplicates within Google dataset:
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']
>>> Count of duplicates: 1181


>>> Examples of duplicates within Apple dataset:
['Mannequin Challenge', 'VR Roller Coaster']
>>> Count of duplicates: 2


In [9]:
for i in google:
    name = i[0]
    if name == "Slack":
        print(i)

['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


Looking at the duplicate data, we see that the rating amount seems to differ between entries of the same application. Here with the example of "Slack". Hence we are going to remove all duplicate entries of the same app, keeping only the one with the highest rating. This logic does only apply to the Google data, the App Store data follows a different logic, where all entries represent a individual App. Apparently Apps within the App Store can have the same name - see [here](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps/discussion/90409) for more.

In [10]:
reviews_max_google = {}

for i in google[1:]:
    name = i[0]
    ratings = float(i[3])
        
    if name in reviews_max_google and reviews_max_google[name] < ratings:
        reviews_max_google[name] = ratings
    elif name not in reviews_max_google:
        reviews_max_google[name] = ratings

print(len(reviews_max_google))

9659


In [11]:
google_clean = []
already_added = []

google_clean.append(google[0])

for row in google[1:]:
    name = row[0]
    ratings = float(row[3])
    
    if ratings == reviews_max_google[name] and name not in already_added:
        google_clean.append(row)
        already_added.append(name)
        
print(len(google_clean))

9660


# Data cleaning 3 - Removing non-english apps

In [12]:
def english_check(string):
    counter = 0
    
    for i in string:
        asci = ord(i)       
        if counter == 3:
            return False
        elif asci >= 127:
            counter += 1
            
    return True

We are going to remove non-english apps, by checking their Name and removing Apps with characters in their names that do not belong to the english language according to the ASCII. TM is outside of the english language range (0-127). Same for emojis hence we are adjusting that a name can contain max 3 characters outside of the ASCII range.

In [13]:
english_check('Instagram')

True

In [14]:
english_check('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [15]:
english_check('Docs To Go™ Free Office Suite') # 

True

In [16]:
english_check('Instachat 😜')

True

Lets iterate over the dataset in order to clean both sets from Apps that should not affect our analysis.

In [17]:
def english_check_loop(dataset, column):
    
    cleaned_set = []
    cleaned_set.append(dataset[0])
    
    for i in dataset[1:]:
        name = i[column]
        
        if english_check(name):
            cleaned_set.append(i)
    
    return cleaned_set

apple_english_only = english_check_loop(apple, 1)
google_english_only = english_check_loop(google_clean, 0)

In [18]:
explore(apple_english_only,0,3)

The data has 6176 rows


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']




In [19]:
explore(google_english_only,0,3)

The data has 9601 rows


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']




# Data cleaning 4 - Removing non-free apps

In [20]:
def price_detector(dataset, column):
    
    cleaned_set = []
    cleaned_set.append(dataset[0])
    
    for i in dataset[1:]:
        price = i[column]
                
        if price == '0' or price == '0.0' or price == 'Free':
            cleaned_set.append(i)
            
    return cleaned_set

google_clean_set = price_detector(google_english_only, 7)
apple_clean_set = price_detector(apple_english_only, 4)

In [21]:
explore(google_clean_set, 0, 5)

The data has 8852 rows


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']




In [22]:
explore(apple_clean_set, 0, 5)

The data has 3217 rows


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']




# Most common Apps by Genre

As our revenue is highly influenced by the number of users, we want to identify apps that are attracting more users on both platforms. We aligned on the following strategy:

1. Build an MVP for the Google Play store
2. If the app has good user responses, it will be developed further
3. If the app is profitable after 6 months, it will be translated for the App store

We will go ahead and inspect the `prime_genre` within the App Store dataset and the `Category` and `Genres` within the Google Play dataset. 

In [23]:
def freq_table(dataset, index):
    
    result = {}
    total = 0
    
    for i in dataset:
        check = i[index]
        total += 1
        
        if check in result:
            result[check] += 1
        else:
            result[check] = 1
            
    for i in result:
        percentage = (result[i] / total)*100
        result[i] = percentage   
        
    return result

In [24]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Running the frequency analysis on the Apple dataset:

In [25]:
display_table(apple_clean_set[1:], 11)

Games : 58.24004975124378
Entertainment : 7.8669154228855716
Photo & Video : 4.975124378109453
Education : 3.6691542288557213
Social Networking : 3.296019900497513
Shopping : 2.5808457711442787
Utilities : 2.4875621890547266
Sports : 2.1455223880597014
Music : 2.0522388059701493
Health & Fitness : 2.0211442786069655
Productivity : 1.7412935323383085
Lifestyle : 1.585820895522388
News : 1.3370646766169154
Travel : 1.2437810945273633
Finance : 1.1194029850746268
Weather : 0.8706467661691543
Food & Drink : 0.808457711442786
Reference : 0.5286069651741294
Business : 0.5286069651741294
Book : 0.404228855721393
Navigation : 0.18656716417910446
Medical : 0.18656716417910446
Catalogs : 0.12437810945273632


The two most common types of Apps in the App Store are `Games` and `Entertainment`. `Photo & Video` and `Education` are following closely. With over half of the Apps being Games (58.22%) the App Store is clearly being domniated by this category. This and seeing that `Entertainment`, `Photo & Video`, `Social Networking` and `Shopping` are relatively high in the ranking leads to the impression that the App Store within the filtered dataset (non-free English apps) is mostly offering Apps for fun and leisure time.

Running the frequency analysis on the Google dataset:

In [26]:
display_table(google_clean_set[1:], 1)

FAMILY : 18.93571347870297
GAME : 9.727714382555645
TOOLS : 8.451022483335215
BUSINESS : 4.598350468873574
PRODUCTIVITY : 3.8978646480623658
LIFESTYLE : 3.8865664896621848
FINANCE : 3.7057959552592923
MEDICAL : 3.5363235792565813
SPORTS : 3.3894475200542313
PERSONALIZATION : 3.321658569653146
COMMUNICATION : 3.2312733024517004
HEALTH_AND_FITNESS : 3.0843972432493505
PHOTOGRAPHY : 2.948819342447181
NEWS_AND_MAGAZINES : 2.801943283244831
SOCIAL : 2.666365382442662
TRAVEL_AND_LOCAL : 2.3387187888374195
SHOPPING : 2.248333521635973
BOOKS_AND_REFERENCE : 2.135351937634166
DATING : 1.8641961360298271
VIDEO_PLAYERS : 1.7964071856287425
MAPS_AND_NAVIGATION : 1.3896734832222348
FOOD_AND_DRINK : 1.242797424019885
EDUCATION : 1.1637103152186195
ENTERTAINMENT : 0.9603434640153655
LIBRARIES_AND_DEMO : 0.937747147215004
AUTO_AND_VEHICLES : 0.9264489888148231
HOUSE_AND_HOME : 0.8021692464128347
WEATHER : 0.7908710880126538
EVENTS : 0.7117839792113886
PARENTING : 0.6552931872104847
ART_AND_DESIGN : 0.

In [27]:
display_table(google_clean_set[1:], 9)

Tools : 8.439724324935035
Entertainment : 6.078409219297255
Education : 5.355327081685686
Business : 4.598350468873574
Productivity : 3.8978646480623658
Lifestyle : 3.875268331262004
Finance : 3.7057959552592923
Medical : 3.5363235792565813
Sports : 3.457236470455316
Personalization : 3.321658569653146
Communication : 3.2312733024517004
Action : 3.106993560049712
Health & Fitness : 3.0843972432493505
Photography : 2.948819342447181
News & Magazines : 2.801943283244831
Social : 2.666365382442662
Travel & Local : 2.327420630437239
Shopping : 2.248333521635973
Books & Reference : 2.135351937634166
Simulation : 2.0449666704327196
Dating : 1.8641961360298271
Arcade : 1.8415998192294656
Video Players & Editors : 1.7738108688283811
Casual : 1.7625127104282001
Maps & Navigation : 1.3896734832222348
Food & Drink : 1.242797424019885
Puzzle : 1.1298158400180771
Racing : 0.9942379392159078
Role Playing : 0.937747147215004
Libraries & Demo : 0.937747147215004
Auto & Vehicles : 0.9264489888148231
St

When looking at the Google Play store, the `Family`, `Games` and `Tools` genre seem to be the most dominating App categories. Comparing the store to the App Store, it seems:

1. much more spread in distribution of # of apps per genre and 
2. overall a bit less focused on pure fun and leisure, 

but showing slightly more `Education` and `Productivity` Apps.

# Most popular Apps by Genre on the App Store

In [28]:
unique_genres_apple = freq_table(apple_clean_set[1:], 11)

In [29]:
print("Genre : Average rating")
print("----------------------")
for genre in unique_genres_apple:
    total = 0
    len_genre = 0
    
    for i in apple_clean_set[1:]:
        genre_app = i[11]
        rating_amount = float(i[5])
        
        if genre_app == genre:
            total += rating_amount
            len_genre += 1
    
    print(genre + " : "  + str(round((total / len_genre),2)) )

Genre : Average rating
----------------------
Social Networking : 71548.35
Photo & Video : 28441.54
Games : 22800.84
Music : 57326.53
Reference : 79350.47
Health & Fitness : 23298.02
Weather : 52279.89
Utilities : 18917.04
Travel : 28243.8
Shopping : 27230.73
News : 21248.02
Navigation : 86090.33
Lifestyle : 16485.76
Entertainment : 14084.89
Food & Drink : 33333.92
Sports : 23008.9
Book : 42816.85
Finance : 31467.94
Education : 7003.98
Productivity : 21028.41
Business : 7491.12
Catalogs : 4004.0
Medical : 612.0


Looking at the average rating numbers as a substitute for user numbers, we have a very different view, compared to previous analysis. In the App Store dataset we do not have the number of installs / downloads available. One more obvious learning: A high amount of numbers in one genre of an app, does not result in a high number of users in that genre.

We see that more practical app genres stand out, hence we could recommend the top three app genres to our developers: `Navigation`, `Reference`, and `Social Networking`.

# Most Popular Apps by Genre on Google Play

In [30]:
display_table(google_clean_set[1:], 5)

1,000,000+ : 15.749632809851994
100,000+ : 11.535419726584568
10,000,000+ : 10.563778104169021
10,000+ : 10.190938876963054
1,000+ : 8.394531691334313
100+ : 6.925771099310812
5,000,000+ : 6.824087673709185
500,000+ : 5.558693932888939
50,000+ : 4.767822844876285
5,000+ : 4.496667043271946
10+ : 3.5363235792565813
500+ : 3.2425714608518814
50,000,000+ : 2.3048243136368773
100,000,000+ : 2.135351937634166
50+ : 1.920686928030731
5+ : 0.7908710880126538
1+ : 0.5084171280081347
500,000,000+ : 0.2711558016043385
1,000,000,000+ : 0.22596316800361538
0+ : 0.045192633600723085
0 : 0.011298158400180771


In [31]:
unique_genres_google = freq_table(google_clean_set[1:], 1)

In [32]:
for genre in unique_genres_google:
    total = 0
    amount = 0
    
    for row in google_clean_set[1:]:
        genre_app = row[1]
        
        if genre == genre_app:
            installs = row[5].replace("+","")
            installs = installs.replace(",","")
            
            total += float(installs)
            amount += 1
            
    print(genre + " : " + str(round((total / amount),2)))

ART_AND_DESIGN : 1986335.09
AUTO_AND_VEHICLES : 647317.82
BEAUTY : 513151.89
BOOKS_AND_REFERENCE : 8814199.79
BUSINESS : 1712290.15
COMICS : 832613.89
COMMUNICATION : 38590581.09
DATING : 854028.83
EDUCATION : 1833495.15
ENTERTAINMENT : 11640705.88
EVENTS : 253542.22
FINANCE : 1387692.48
FOOD_AND_DRINK : 1924897.74
HEALTH_AND_FITNESS : 4188821.99
HOUSE_AND_HOME : 1360598.04
LIBRARIES_AND_DEMO : 638503.73
LIFESTYLE : 1446158.22
GAME : 15606004.01
FAMILY : 3695641.82
MEDICAL : 120550.62
SOCIAL : 23253652.13
SHOPPING : 7036877.31
PHOTOGRAPHY : 17840110.4
SPORTS : 3650602.28
TRAVEL_AND_LOCAL : 13984077.71
TOOLS : 10830251.97
PERSONALIZATION : 5201482.61
PRODUCTIVITY : 16787331.34
PARENTING : 542603.62
WEATHER : 5145550.29
VIDEO_PLAYERS : 24727872.45
NEWS_AND_MAGAZINES : 9549178.47
MAPS_AND_NAVIGATION : 4049274.63


Looking at the Google Play store data, the following genres sound promissing, due to high install numbers: `TOOLS`, `GAME`, `BOOKS_AND_REFERENCE` and `NEWS_AND_MAGAZINES`. If we combine the insights from the Apple Store, we can recommend the `Reference` (Apple) or `BOOKS_AND_REFERENCE` (Google) category to our developers. 

We can recomend to translate a book into an interactive app which triggers the user to stay within the app and regularly come back for news and updates.